# RAG: Function Calling

OpenAI provides an inbuilt mechanism to achieve function calling.
We will try to replicate the same with some local models so that
you can try function calling without openAI.

# Set up

In [1]:
!pip install --upgrade --quiet beautifulsoup4 langchain langchain_community langchain_core langchain_groq langchain_text_splitters sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 735.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3

In [51]:
import os
import re
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

I used the google colab to create this pipeline. so its recommended to use colab secrets to store the credentials.

we would be required to create account at

```Pinecone
Groq
Langsmith
openai
```
you have to copy the api keys of all the above account to the secrets. the api keys can be fetched using


```

from google.colab import userdata
secret = userdata.get('secret-key')
make sure to have api keys ready.
```

In [5]:
from google.colab import userdata
groq_key = userdata.get('groq-key')
smith_key = userdata.get('smith-key')
pine_key = userdata.get('pine-key')
openai_key = userdata.get('openai-key')

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'rag-function-calling'
os.environ['LANGCHAIN_API_KEY'] = smith_key
os.environ['GROQ_API_KEY'] = groq_key

# Use case - Flight booking based on the prompt given by user.

Objective is that
 - the application should understand the user query, and
 - should direct the required information to the booking function (a dummy function that is intended to do the booking).


# Function Calling using langchain.agents

Lets set the prompt which will make the system understand what it is supposed to do.

In [107]:
template = """The user wants to book a flight. Extract the source, destination and date from the query. \n
              you first, Format the response as 'source: [source], destination: [destination], date: [date]'.\n
              once you get the source and destination, replace them with airport standard code, dont explain the changes in your response. \n
              Query: {query}
            """
prompt_template = ChatPromptTemplate.from_template(template)

In [108]:
# Function to parse the destination and date from the structured response
def parse_flight_details(output):
    match_group = re.search(r"source: ([A-Za-z\s]+), destination: ([A-Za-z\s]+), date: ([A-Za-z0-9\s]+)", output)

    if match_group:
        source = match_group.group(1)
        destination = match_group.group(2)
        date = match_group.group(3)
        return f"Flight booked from {source} to {destination} on {date}."
    else:
        return "Could not extract complete details from the response."

In [109]:
# Function that calls the language model and uses the structured prompt
def book_flight(query: str):
    llm = ChatGroq(temperature=0)  # You can replace it another LLM

    generate_queries_chain = (
      prompt_template
      | llm
      | StrOutputParser()
    )

    # Call the model with the prompt and get the output
    output = generate_queries_chain.invoke(query)

    print(f"output after prompt generation: {output}")
    # Parse the structured output to extract destination and date
    return parse_flight_details(output)

In [110]:
# Create a tool that wraps the function with structured output
tools = [
    Tool(
        name="book_flight",
        func=book_flight,  # Pass the full query to the function
        description="Book a flight by providing Source, destination and date in the format: 'from [Source] to [destination] on [date]'."
    )
]

In [111]:
# Initialize an agent with a language model
llm = ChatGroq(temperature=0)  # You can use other models like HuggingFace here
agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm
)

In [112]:
# Define a user query to invoke the agent
user_query = "Book a flight from Delhi to New York on October 20."

# Call the agent with the query
result = agent.run(user_query)

output after prompt generation: source: Delhi, destination: New York, date: October 20
source: DEL, destination: JFK, date: October 20


In [113]:
# Output the result
print(result)

Your flight from Delhi to New York on October 20 has been booked.


# Function calling using Structured LLMs

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough

In [116]:
class BookMyFlight(BaseModel):
    """Book your flight using the chat assistance."""

    query: str = Field(
        ...,
        description="Book a flight by providing Source, destination and date in the format: 'from [Source] to [destination] on [date]'.",
    )
    source: str = Field(
        ...
        , description="Source location"
    )
    destination: str = Field(
        ...
        , description="destination location"
    )
    date: Optional[str] = Field(
        ...
        , description="date of travel"
    )

In [117]:
system = """
  The user wants to book a flight. Extract the source, destination and date from the query. \n
  you first, Format the response as 'source: [source], destination: [destination], date: [date]'.\n
  once you get the source and destination, replace them with airport standard code, dont explain the changes in your response. \n
"""

In [119]:
llm = ChatGroq(temperature=0)

structured_llm = llm.with_structured_output(BookMyFlight)


prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

query_analyzer = {"question": RunnablePassthrough()} | prompt_template | structured_llm



In [121]:
def book_flights(output: BookMyFlight):
    return f"Flight booked from {output.source} to {output.destination} on {output.date}."


In [120]:
# Call the model with the prompt and get the output
query = "Book a flight from Delhi to New York on October 20."
output = query_analyzer.invoke(query)
output

BookMyFlight(query='Book a flight from Delhi to New York on October 20', source='Delhi', destination='New York', date='October 20')

In [122]:
book_flights(output)

'Flight booked from Delhi to New York on October 20.'